In [1]:
import os

from vertexai.preview.language_models import TextGenerationModel

os.environ[
    "GOOGLE_APPLICATION_CREDENTIALS"
] = "credentials/aiap-14-ds-llm-topic-01-sa.json"

model = TextGenerationModel.from_pretrained("text-bison@001")

In [2]:
!pip install langchain-experimental
!pip install termcolor
!pip install langchain-experimental
!pip install tik-token

ERROR: Could not find a version that satisfies the requirement tik-token (from versions: none)
ERROR: No matching distribution found for tik-token


In [2]:
import faiss

from datetime import datetime, timedelta
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import VertexAIEmbeddings

In [3]:
import logging

logging.basicConfig(level=logging.ERROR)

In [4]:
from datetime import datetime, timedelta
from typing import List
from termcolor import colored


from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS

In [5]:
import math
from src.generative_agents.generative_agent import StemssGenerativeAgent
from src.generative_agents.memory import StemssGenerativeAgentMemory
from src.retrievers.time_weighted_retriever import ModTimeWeightedVectorStoreRetriever
from src.vectorstores.chroma import EnhancedChroma


def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return 1.0 - score / math.sqrt(2)


def create_new_memory_retriever(decay_rate: float = 0.5, k: int = 5):
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = VertexAIEmbeddings()
    vs = EnhancedChroma(embedding_function=embeddings_model)
    return ModTimeWeightedVectorStoreRetriever(
        vectorstore=vs, other_score_keys=["importance"], decay_rate=decay_rate, k=k
    )

In [6]:
from langchain.document_loaders import CSVLoader

mem_file_joel = "./memory/memory.csv"
mem_file_ellie = "./memory/memory_ellie.csv"


def load_documents(mem_file):
    """ "Load memory (history) from CSV "

    Returns
    -------
    list
        A list of documents where each row of the CSV is a document
    """
    loader = CSVLoader(
        mem_file, metadata_columns=["last_accessed_at", "created_at", "importance"]
    )
    docs = loader.load()
    return docs

In [7]:
docs = load_documents(mem_file=mem_file_joel)
memory_retriever_joel = create_new_memory_retriever()
memory_retriever_joel.add_documents(docs)

docs = load_documents(mem_file=mem_file_ellie)
memory_retriever_ellie = create_new_memory_retriever()
memory_retriever_ellie.add_documents(docs)

['90aed9f1-69a9-11ee-b87d-106fd9de73c8',
 '90aed9f2-69a9-11ee-b912-106fd9de73c8',
 '90aed9f3-69a9-11ee-ba7b-106fd9de73c8',
 '90aed9f4-69a9-11ee-9499-106fd9de73c8',
 '90aed9f5-69a9-11ee-82aa-106fd9de73c8',
 '90aed9f6-69a9-11ee-9ac4-106fd9de73c8',
 '90aed9f7-69a9-11ee-94b0-106fd9de73c8',
 '90aed9f8-69a9-11ee-9ebd-106fd9de73c8',
 '90aed9f9-69a9-11ee-9b9d-106fd9de73c8',
 '90aed9fa-69a9-11ee-9d69-106fd9de73c8',
 '90aed9fb-69a9-11ee-8c32-106fd9de73c8']

In [17]:
from langchain.llms import VertexAI
from langchain.prompts import ChatPromptTemplate

USER_NAME = "Person A"  # The name you want to use when interviewing the agent.
llm = VertexAI(model_name="text-bison@001", max_output_tokens=256, temperature=0.5)

In [18]:
from src.generative_agents.generative_agent import StemssGenerativeAgent
from src.generative_agents.memory import StemssGenerativeAgentMemory

joel_memory = StemssGenerativeAgentMemory(
    llm=llm,
    memory_retriever=memory_retriever_joel,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
    verbose=True,
)

joel = StemssGenerativeAgent(
    name="Joel",
    age=25,
    traits="curious, enthusiastic, paranoid",  # You can add more persistent traits here
    status="going home for his birthday",  # When connected to a virtual world, we can have the characters update their status
    memory_retriever=create_new_memory_retriever(),
    llm=llm,
    memory=joel_memory,
    background="Joel is a music lover with a knack for composing sad songs. Joel has posted many songs on YouTube and has gotten quite some views.",
)

In [10]:
print(joel.get_summary())

Name: Joel (age: 25)
Innate traits: curious, enthusiastic, paranoid
Joel is a music lover who composes sad songs. He is also a good cook and enjoys watching movies.


In [19]:
ellie_memory = StemssGenerativeAgentMemory(
    llm=llm,
    memory_retriever=memory_retriever_ellie,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
    verbose=True,
)

ellie = StemssGenerativeAgent(
    name="Ellie",
    age=52,
    traits="tired, hungry, well-read",  # You can add more persistent traits here
    status="finishing up retirement procedures",  # When connected to a virtual world, we can have the characters update their status
    memory_retriever=create_new_memory_retriever(),
    llm=llm,
    memory=ellie_memory,
    background="Ellie has a 5 Phds and has been in Academia for 30 years. Ellie wants to travel the world and do voluntary work.",
)

In [12]:
print(ellie.get_summary())

Name: Ellie (age: 52)
Innate traits: tired, hungry, well-read
Ellie is a 50-year-old woman who has been in academia for 30 years. She has a Ph.D. in five different fields and is interested in traveling the world and doing voluntary work. She is married to Joel and they have a good relationship. Ellie is a kind and caring person who is always willing to help others. She is also very intelligent and creative.


In [13]:
def run_conversation(
    agents: List[StemssGenerativeAgent], initial_observation: str
) -> None:
    """Runs a conversation between agents."""
    _, observation = agents[1].generate_reaction(initial_observation)
    print(observation)
    turns = 0
    while True:
        break_dialogue = False
        for agent in agents:
            # print(agent.generate_dialogue(, observation))
            stay_in_dialogue, observation = agent.generate_dialogue(
                agent.name, observation
            )
            # observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                break_dialogue = True
        if break_dialogue:
            break
        turns += 1

In [25]:
agents = [joel, ellie]
run_conversation(
    agents,
    "Joel said: Hi, Ellie. Thanks for agreeing to meet with me today. I have this paper that I have to complete tomorrow on the history of America. Could you teach me how to do it?",
)



> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is not important at all (e.g., brushing teeth, making bed) and 10 is extremely important (e.g., a break up, college acceptance), rate the importance of the following piece of memory. You must respond with a single integer.
Memory: Ellie observed Joel said: Hi, Ellie. Thanks for agreeing to meet with me today. I have this paper that I have to complete tomorrow on the history of America. Could you teach me how to do it? and reacted by SAY: Of course, Joel. I'd be happy to help. Let's get started.
Rating: 

> Finished chain.
Ellie said Of course, Joel. I'd be happy to help. Let's get started.
PROMPT: input_variables=['agent_name', 'agent_status', 'agent_summary_description', 'current_time', 'most_recent_memories', 'observation', 'relevant_memories'] template='{agent_summary_description}\nIt is {current_time}.\n{agent_name}\'s status: {agent_status}\nSummary of relevant context from {agent_name}\'s

In [15]:
joel.summarize_speaker_memories("Joel is going to the park and sees a chihuahua")

'The entity in this observation is "Joel".\n\nThe observation is about Joel going to the park and seeing a chihuahua. The entity is the person who is doing the action. In this case, the person is Joel.\n\nThe relationship between Joel and The entity in this observation is "Joel" is that Joel is the person who is doing the action.'

In [23]:
ellie_observations = [
    "ellie wakes up and hear's the alarm",
    "ellie eats a boal of porridge",
    "ellie helps a coworker on a task",
    "ellie plays tennis with her friend Xu before going to work",
    "ellie overhears her colleague say something about Tommie being hard to work with",
]
for observation in ellie_observations:
    ellie.memory.add_memory(observation)



> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is not important at all (e.g., brushing teeth, making bed) and 10 is extremely important (e.g., a break up, college acceptance), rate the importance of the following piece of memory. You must respond with a single integer.
Memory: ellie wakes up and hear's the alarm
Rating: 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is not important at all (e.g., brushing teeth, making bed) and 10 is extremely important (e.g., a break up, college acceptance), rate the importance of the following piece of memory. You must respond with a single integer.
Memory: ellie eats a boal of porridge
Rating: 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is not important at all (e.g., brushing teeth, making bed) and 10 is extremely important (e.g., a break up, college acceptance), rate the